In [18]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub

In [3]:
mlflow.set_tracking_uri('https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow')
dagshub.init(repo_owner='Abhaysharma53', repo_name='mlops-mini-project', mlflow=True)

Accessing as Abhaysharma53

Initialized MLflow to track repo "Abhaysharma53/mlops-mini-project"

Repository Abhaysharma53/mlops-mini-project initialized!

In [4]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

In [6]:
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    text = [word for word in str(text).split() if word not in stop_words ]
    return " ".join(text)
    

In [7]:
def removing_numbers(text):
    text = ''.join([char for char in text if not char.isdigit()])
    return text

In [8]:
def lower_case(text):
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

In [9]:
def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [10]:
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [11]:
#normalize the text data
df = normalize_text(df)

In [12]:
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [13]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [20]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Set the experiment name
mlflow.set_experiment("LR Hyperparameter Tuning")

2024/11/10 21:54:18 INFO mlflow.tracking.fluent: Experiment with name 'LR Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/485ba4aa21bc4e2b8caf3687c725dba2', creation_time=1731255858456, experiment_id='3', last_update_time=1731255858456, lifecycle_stage='active', name='LR Hyperparameter Tuning', tags={}>

In [16]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}


In [22]:
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook
    #mlflow.log_artifact(__file__)
    import os
    notebook_path = "C:\\Users\\admin\\Desktop\\DSMP\\mlops-mini-project\\mlops-mini-project\\notebooks\\exp_3_LR_BOW_HP.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

Mean CV Score: 0.7050121790562889, Std CV Score: 0.014034901421846473
Accuracy: 0.739277108433735
Precision: 0.7749419953596288
Recall: 0.6581280788177339
F1 Score: 0.7117741076185403


2024/11/10 22:49:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3/runs/c08df87537604db7a2736dc2d38d9de2.
2024/11/10 22:49:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3.


Mean CV Score: 0.7799418860816888, Std CV Score: 0.011174824136692792
Accuracy: 0.7898795180722892
Precision: 0.7818889970788705
Recall: 0.7911330049261084
F1 Score: 0.7864838393731637


2024/11/10 22:50:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3/runs/161d17334cdd40c68da0b7a5694ac4b4.
2024/11/10 22:50:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3.


Mean CV Score: 0.7832047930450088, Std CV Score: 0.008669249705645539
Accuracy: 0.7826506024096386
Precision: 0.7737864077669903
Recall: 0.7852216748768472
F1 Score: 0.7794621026894866


...

2024/11/10 22:50:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3/runs/edb8c7195f6c45d1a52dfbc014bba35e.
2024/11/10 22:50:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3.


Mean CV Score: 0.7806493249611147, Std CV Score: 0.004790409981330222
Accuracy: 0.7826506024096386
Precision: 0.7706333973128598
Recall: 0.7911330049261084
F1 Score: 0.7807486631016043


2024/11/10 22:50:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3/runs/3b8a1cd916274ef79bd02f572215c7a2.
2024/11/10 22:50:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3.


Best Params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 Score: 0.7896190577231609


2024/11/10 22:51:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/10 22:51:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-shrike-898 at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3/runs/a164d08859724bc79679b59b93677511.
2024/11/10 22:51:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Abhaysharma53/mlops-mini-project.mlflow/#/experiments/3.


# Best set of parameter -> C = 1 & solver = liblinear & penalty = L2
